In [1]:
import os
import time
import datetime
import logging
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
import random

from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2.data import DatasetCatalog, DatasetMapper, build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.modeling import BACKBONE_REGISTRY, Backbone, ShapeSpec
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.utils.events import EventStorage
from detectron2.checkpoint import DetectionCheckpointer
import timm
import sys
# 추가적으로 필요한 import
from detectron2.evaluation import DatasetEvaluators
import logging
logging.getLogger("detectron2").setLevel(logging.WARNING)
logging.getLogger("detectron2").disabled = True

@BACKBONE_REGISTRY.register()
class TimmBackbone(Backbone):
    def __init__(self, cfg, input_shape):
        super().__init__()
        model_name = cfg.MODEL.BACKBONE.TIMM_MODEL
        self.model = timm.create_model(model_name, features_only=True, pretrained=True)
        feature_info = self.model.feature_info.get_dicts(keys=['num_chs', 'reduction'])
        
        self.out_channels = 256
        self.convs = nn.ModuleDict()
        for i, info in enumerate(feature_info):
            self.convs[f"p{i+2}"] = nn.Conv2d(info['num_chs'], self.out_channels, kernel_size=1)
        
        # P6, P7 레벨 추가 (RetinaNet용)
        self.p6 = nn.Conv2d(feature_info[-1]['num_chs'], self.out_channels, kernel_size=3, stride=2, padding=1)
        self.p7 = nn.Conv2d(self.out_channels, self.out_channels, kernel_size=3, stride=2, padding=1)
        
        self._out_features = ["p2", "p3", "p4", "p5", "p6", "p7"]
        self._out_feature_channels = {name: self.out_channels for name in self._out_features}
        self._out_feature_strides = {f"p{i+2}": info['reduction'] for i, info in enumerate(feature_info)}
        self._out_feature_strides["p6"] = self._out_feature_strides["p5"] * 2
        self._out_feature_strides["p7"] = self._out_feature_strides["p6"] * 2

    def forward(self, x):
        features = self.model(x)
        out = {f"p{i+2}": self.convs[f"p{i+2}"](feature) for i, feature in enumerate(features)}
        p6 = self.p6(features[-1])
        p7 = self.p7(F.relu(p6))
        out["p6"] = p6
        out["p7"] = p7
        return out

    def output_shape(self):
        return {
            name: ShapeSpec(
                channels=self._out_feature_channels[name], stride=self._out_feature_strides[name]
            )
            for name in self._out_features
        }

def setup_cfg(model_type):
    cfg = get_cfg()
    if model_type == "Cascade R-CNN":
        cfg.merge_from_file(model_zoo.get_config_file("Misc/cascade_mask_rcnn_R_50_FPN_3x.yaml"))
        cfg.MODEL.FPN.IN_FEATURES = ["p2", "p3", "p4", "p5", "p6"]
    elif model_type == "RetinaNet":
        print("??")
        cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_50_FPN_3x.yaml"))
        cfg.MODEL.FPN.IN_FEATURES = ["p2", "p3", "p4", "p5", "p6", "p7"]
    else:  # FPN-based Faster R-CNN
        cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
        cfg.MODEL.FPN.IN_FEATURES = ["p2", "p3", "p4", "p5", "p6"]
    print("?")
    cfg.DATASETS.TRAIN = ("my_dataset_train",)
    cfg.DATASETS.TEST = ("my_dataset_val",)

    cfg.MODEL.BACKBONE.NAME = "TimmBackbone"
    cfg.MODEL.BACKBONE.TIMM_MODEL = "resnet50"    
    
    cfg.MODEL.FPN.OUT_CHANNELS = 256     

    cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[32], [64], [128], [256], [512]]    
    cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.5, 1.0, 2.0]]

    cfg.MODEL.RPN.IN_FEATURES = cfg.MODEL.FPN.IN_FEATURES

    cfg.MODEL.WEIGHTS = ""
    
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10
    cfg.INPUT.MIN_SIZE_TRAIN = (1024,)
    cfg.INPUT.MAX_SIZE_TRAIN = 1024
    cfg.INPUT.MIN_SIZE_TEST = 1024
    cfg.INPUT.MAX_SIZE_TEST = 1024
    cfg.SOLVER.IMS_PER_BATCH = 4
    cfg.SOLVER.BASE_LR = 0.003
    # epoch 설정
    dataset_size = 4883
    num_epochs = 1  # 원하는 epoch 수
    iterations_per_epoch = dataset_size // cfg.SOLVER.IMS_PER_BATCH
    cfg.SOLVER.MAX_ITER = iterations_per_epoch * num_epochs
    cfg.TEST.EVAL_PERIOD = num_epochs

    # cuDNN 벤치마크 비활성화
    # torch.backends.cudnn.enabled = False
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128

    # 학습률 스케줄 조정 (선택사항)
    #cfg.SOLVER.STEPS = (iterations_per_epoch * 30, iterations_per_epoch * 40) 

    cfg.MODEL.MASK_ON = False

    return cfg
    
image_dir = '../../dataset'
json_file_path = '../../dataset/train.json'

# 데이터셋 등록 (COCO 형식 가정)
register_coco_instances("my_dataset_train", {}, image_dir + "/train.json", image_dir)
register_coco_instances("my_dataset_val", {}, image_dir + "/test.json", image_dir)



In [2]:

def sample_train_data(dataset_dicts, sample_ratio=0.2):
    """학습 데이터의 일부를 샘플링합니다."""
    sampled_data = random.sample(dataset_dicts, int(len(dataset_dicts) * sample_ratio))
    return sampled_data

def evaluate_on_train_set(cfg, model, dataset_name):
    dataset_dicts = DatasetCatalog.get(dataset_name)
    sampled_data = sample_train_data(dataset_dicts)
    
    mapper = DatasetMapper(cfg, is_train=False)
    sampled_loader = build_detection_test_loader(sampled_data, mapper=mapper)
    
    evaluator = COCOEvaluator(dataset_name, cfg, False, output_dir=cfg.OUTPUT_DIR)

    # tqdm을 사용하여 평가 진행 상황 표시
    results = inference_on_dataset(model, sampled_loader, evaluator)
   
    # 'bbox' 키에서 클래스별 AP와 AR 값을 추출
    class_metrics = {}
    for class_id, metrics in results['bbox'].items():
        if isinstance(class_id, int):  # 클래스 ID인 경우만 처리
            class_metrics[class_id] = {
                'AP': metrics['AP'],
                'AR': metrics['AR']
            }
    
    return class_metrics

def calculate_mAP(metrics):
    ap_values = [values['AP'] for class_id, values in metrics.items() if isinstance(class_id, int)]
    return sum(ap_values) / len(ap_values) if ap_values else 0

def train_model(cfg, model_name):
    trainer = DefaultTrainer(cfg)
    trainer.resume_or_load(resume=False)

    # 모델명에 맞는 디렉토리 생성
    model_output_dir = os.path.join(cfg.OUTPUT_DIR, model_name)
    os.makedirs(model_output_dir, exist_ok=True)
    cfg.OUTPUT_DIR = model_output_dir

    checkpointer = DetectionCheckpointer(trainer.model, save_dir=cfg.OUTPUT_DIR)

    loss_window = 20  # 최근 20개 iteration의 평균 손실을 계산합니다
    eval_period = 1000  # 1000 iteration마다 평가를 수행합니다
    save_period = 3000  # 5000 iteration마다 모델을 저장합니다
    best_mAP = 0.

    with EventStorage() as storage:
        with tqdm(total=cfg.SOLVER.MAX_ITER, desc=f"Training {model_name}") as pbar:
            for iteration in range(cfg.SOLVER.MAX_ITER):
                trainer.run_step()
                mAP = 0.
                if (iteration + 1) % 2 == 0:
                    losses = storage.histories()["total_loss"].avg(loss_window)
                    lr = trainer.optimizer.param_groups[0]["lr"]
                    pbar.set_postfix({'loss': f'{losses:.4f}', 'lr': f'{lr:.6f}'})
                
                if (iteration + 1) % eval_period == 0:
                    pbar.set_description(f"Evaluating {model_name}")
                    metrics = evaluate_on_train_set(cfg, trainer.model, cfg.DATASETS.TRAIN[0])

                    mAP = calculate_mAP(metrics)
                    best_mAP = max(best_mAP, mAP)

                    class_metrics = []
                    for class_id, values in metrics.items():
                        ap = values['AP']
                        ar = values['AR']
                        class_metrics.append(f"C{class_id}: AP:{ap:.2f}, AR:{ar:.2f}")
                    
                    class_metrics_str = " | ".join(class_metrics)
                    pbar.set_description(f"Training {model_name} | {class_metrics_str}")
                
                # 특정 iteration마다 모델 저장
                if (iteration + 1) % save_period == 0:
                    checkpointer.save(f"{model_name}_iter_{iteration+1}_mAP_{mAP:.4f}")
                    print(f"\nSaved model at iteration {iteration+1}")
                
                pbar.update(1)

    # 학습이 끝난 후 최종 모델 저장
    checkpointer.save(f"{model_name}_final_mAP_{best_mAP:.4f}")
    print(f"\nSaved final model for {model_name}")

    print(f"{model_name} 학습 완료")


sys.path
cfg_retinanet = setup_cfg("RetinaNet")
# train_model(cfg_retinanet, "RetinaNet")
# # Set up configurations and train models
# cfg_fpn = setup_cfg("FPN-based Faster R-CNN")
# train_model(cfg_fpn, "FPN-based Faster R-CNN")

# cfg_cascade = setup_cfg("Cascade R-CNN")
# train_model(cfg_cascade, "Cascade R-CNN")






??


RuntimeError: COCO-Detection/retinanet_R_50_FPN_3x.yaml not available in Model Zoo!